# INSTALL
- WARNING ::: ONLY DO ONCE 
    - update devconfig in ~/.config/pyontutils/devonfig.yaml
        - scigraph_api: http://scigraph.scicrunch.io:9000/scigraph
    - Install both pyontutils and ilxutils with pyontutils
        - cd ~/git/pyontutils
        - pip3 install --user --editable .
        - cd ~/git/pyontutils/ilxutils/
        - pip3 install --user --editable .
    - Clone ontquery and install
        - cd ~/git
        - git clone https://github.com/tgbugs/ontquery.git
        - cd ~/git/ontquery
        - pip3 install --user --editable .

# Maintainance
- update repos
    - cd ~/git/pyontutils
    - git pull 
    - cd ~/git/ontquery
    - git pull

# Google Sheets Import
### Need pyontutils secrets.yaml setup first!

In [4]:
from pyontutils.sheets import Sheet
import pandas as pd
KEY_NAME = 'sparc-terms'
SHEET_NAME = 'Minimal information model(MIS)'

class Brainstem(Sheet):
    name = KEY_NAME # key name you gave the google sheet id value in secrets.yaml
    sheet_name = SHEET_NAME # the actual sheet name on the google sheet
    fetch_grid = True # meta data in self.grid that has detials like bolding

brainstem = Brainstem()
df = pd.DataFrame(brainstem.raw_values)
df.columns = df.iloc[0]
df.drop(df.index[0], inplace=True)
df.head()

In [8]:
list(df.term)[:10]

['abdominal cavity',
 'abdominal wall',
 'adipose tissue',
 'adult organism',
 'alimentary part of gastrointestinal system',
 'arterial blood',
 'biceps femoris',
 'blood',
 'bolus of food',
 'brainstem']

# CSV or TSV  EXAMPLE

# QUERY DATABASES 

In [5]:
# Give "query" a usable parameter to query the databases 
from pyontutils.core import query # OntTerm
query(term='brain')

[OntTerm('UBERON:0000955', label='brain'),
 OntTerm('UBERON:6110636', label='adult cerebral ganglion')]

In [11]:
# similar entities will show
# default limit is 10
query(term='brain', limit=10, prefix=('ILX')) 

[]

In [12]:
# Faster and more accurate with curie/iri
query(curie='UBERON:0000955')

[OntTerm('UBERON:0000955', label='brain')]

In [13]:
entity = query(curie='UBERON:0000955')[0]
# Full result attribute
vars(entity)

{'prefix': 'UBERON',
 'suffix': '0000955',
 'label': 'brain',
 'labels': ['brain'],
 'definition': 'The brain is the center of the nervous system in all vertebrate, and most invertebrate, animals. Some primitive animals such as jellyfish and starfish have a decentralized nervous system without a brain, while sponges lack any nervous system at all. In vertebrates, the brain is located in the head, protected by the skull and close to the primary sensory apparatus of vision, hearing, balance, taste, and smell[WP].',
 'synonyms': ['the brain',
  'synganglion',
  'suprasegmental structures',
  'suprasegmental levels of nervous system',
  'encephalon'],
 'deprecated': False,
 'predicates': {},
 '_type': OntId('owl:Class'),
 '_types': (OntId('owl:Class'),),
 '_graph': None,
 '_source': <ontquery.plugins.services.scigraph.SciGraphRemote at 0x11b6a0890>,
 'validated': True,
 '_query_result': QueryResult({'iri': 'http://purl.obolibrary.org/obo/UBERON_0000955', 'curie': 'UBERON:0000955', 'label':

# DEBUGGING HINT
- 1 "?" at the end of a function or class will return its params, docstring, and pathing. 
- 2 "??" returns the ENTIRE class/functions 

In [14]:
query?

Signature:      query(*args, **kwargs)
Call signature:
query(
    term=None,
    prefix=(),
    category=None,
    label=None,
    abbrev=None,
    search=None,
    suffix=None,
    curie=None,
    iri=None,
    predicates=(),
    exclude_prefix=(),
    depth=1,
    direction='OUTGOING',
    limit=10,
    include_deprecated=False,
    include_supers=False,
    include_all_services=False,
    raw=False,
)
Type:           OntQueryCli
String form:    <ontquery.query.OntQueryCli object at 0x11b633f90>
File:           ~/Dropbox/git/ontquery/ontquery/query.py
Docstring:      <no docstring>


# BONUS!

### Concurrently search! (Run multiple query functions at the same time)

In [16]:
from pyontutils.utils import Async, deferred
from pyontutils.core import OntTerm, ixr, query
from typing import List, Tuple

# query.setup()

def queries(kwargs_list:List[dict]) -> List[Tuple[str, dict]]:
    '''Asynchronously query databases to dramatically increase runtime un users end 
    
    Examples:
        >>> queries([{'term':'Brain'},])
        [({'term': 'Brain'},
          [OntTerm('HBA:3999', label='brain (hba)'),
           OntTerm('FMA:50801', label='Brain'),
           OntTerm('UBERON:0000955', label='brain'),
           OntTerm('UBERON:6110636', label='adult cerebral ganglion')])]
        >>> queries([{'curie':'UBERON:0000955'},])
        [({'curie': 'UBERON:0000955'}, [OntTerm('UBERON:0000955', label='brain')])]
        
    Definitions:
        kwargs == common name given to dictionary input for function
        tuple == a list that you cannot update. 
        lambda == short-hand for single line function creation (func = lambda <input>: <output>)  
        
    Args:
        kwargs_list (list): A list of dictionaries that are paramaters for the query function
    
    Returns:
        List[tuple]: A list of tuples all being of (<input>, <query_result>).    
    '''
    # create a query function wrapper to return tuple
    # kwargs -> (kwargs, query_result)
    # We do this in case 2+ queries return the same results & the output WILL NOT have the same input order
    gin = lambda kwargs: (kwargs, query(**kwargs))
    # run each query instance at the same time
    results = Async()(deferred(gin)(kwargs) for kwargs in kwargs_list)
    return results 

queries([{'curie':'UBERON:0000955'}, {'curie':'UBERON:6110636'}])

Futures compiled


[({'curie': 'UBERON:0000955'}, [OntTerm('UBERON:0000955', label='brain')]),
 ({'curie': 'UBERON:6110636'},
  [OntTerm('UBERON:6110636', label='adult cerebral ganglion')])]

# Modifing TEST InterLex

In [20]:
# TEST InterLex endpoints
from ilxutils.remotes import remote
ixrt = remote(server='test3')

# GET ENTITY

In [21]:
ixrt.ilx_cli.get_entity('tmp_0738390')

{'id': '661544',
 'orig_uid': '34142',
 'uid': '34142',
 'orig_cid': '0',
 'cid': '0',
 'ilx': 'ilx_0738390',
 'label': 'official test label',
 'type': 'term',
 'definition': 'definition',
 'comment': 'helpful misc',
 'version': '2',
 'status': '0',
 'display_superclass': '1',
 'orig_time': '1564695195',
 'time': '1564695333',
 'synonyms': [{'id': '1776589',
   'tid': '661544',
   'literal': 'Encephalon',
   'type': '',
   'time': '1564695333',
   'version': '2'},
  {'id': '1776590',
   'tid': '661544',
   'literal': 'Cerebro',
   'type': '',
   'time': '1564695333',
   'version': '2'}],
 'superclasses': [],
 'existing_ids': [{'id': '3885425',
   'tid': '661544',
   'curie': 'ILX:0738390',
   'iri': 'http://uri.interlex.org/base/ilx_0738390',
   'curie_catalog_id': '3885424',
   'version': '2',
   'time': '1564695334',
   'preferred': '1'}],
 'relationships': [],
 'mappings': [],
 'annotations': [],
 'ontologies': []}

# ADD

In [22]:
ixrt.add_entity?

Signature:
ixrt.add_entity(
    type,
    subThingOf,
    label,
    definition: str = None,
    synonyms=(),
    comment: str = None,
    predicates: dict = None,
    existing_ids=None,
    cid=None,
)
Docstring: <no docstring>
File:      ~/Dropbox/git/ontquery/ontquery/plugins/services/interlex.py
Type:      method


In [23]:
entity = dict(
    label = 'offical label', # Can only one unique label per person
    type = 'term', # OPTIONS: term, annotation, relationship, cde, fde, pde
    definition = 'official definition',
    comment = 'helpful misc',
    # Optional
    subThingOf = '', # WARNING ::: must have at last '', can be blank but please fill this in if you can. 
    synonyms = ['Encephalon', 'Cerebro'],
    predicates = {} # annotations and/or relationships to add
    # TODO: existing_ids will be an option later
)
result = ixrt.add_entity(**entity)
print(result)

QueryResult({'iri': 'http://uri.interlex.org/base/ilx_0738390', 'curie': 'ILX:0738390', 'label': 'official test label', 'labels': (), 'definition': 'definition', 'synonyms': ('Encephalon', 'Cerebro'), 'deprecated': None, 'predicates': {'comment': 'helpful misc'}, 'type': None, 'types': (), '_graph': None, 'source': <ontquery.plugins.services.interlex.InterLexRemote object at 0x11ba30b10>})


# UPDATE

In [24]:
ixrt.update_entity?

Signature:
ixrt.update_entity(
    ilx_id: str = None,
    type: str = None,
    subThingOf: str = None,
    label: str = None,
    definition: str = None,
    synonyms=(),
    comment: str = None,
    predicates_to_add: dict = None,
    add_existing_ids: List[dict] = None,
    delete_existing_ids: List[dict] = None,
    predicates_to_delete: dict = None,
    cid: str = None,
)
Docstring:
Update existing entity.

:param List[dict] add_existing_ids: iris and curies to be added to entity.
:param List[dict] delete_existing_ids: iris and curies to be deleted from entity.

>>>update_entity(add_existing_ids=[{'ilx_id':'ilx_1234567', 'iri':'http://abc.org/abc_123', 'curie':'ABC:123'}])
>>>update_entity(delete_existing_ids=[{'ilx_id':'ilx_1234567', 'iri':'http://abc.org/abc_123', 'curie':'ABC:123'}])
File:      ~/Dropbox/git/ontquery/ontquery/plugins/services/interlex.py
Type:      method


In [26]:
from ilxutils.remotes import remote
ixrt = remote(server='test3')
entity = dict(
    ilx_id = 'ILX:0738390',
    label = 'Offical label', # Can only one unique label per person
    type = 'term', # OPTIONS: term, annotation, relationship, cde, fde, pde
    definition = 'official definition',
    comment = 'helpful misc',
    # Optional
    subThingOf = '', # WARNING ::: must have at last '', can be blank but please fill this in if you can. 
    synonyms = ['Encephalon', 'Cerebro'],
    predicates_to_add = {}, # annotations and/or relationships to add
    predicates_to_delete = {}, # annotations and/or relationships to del
    # TODO: existing_ids will be an option later
)
result = ixrt.update_entity(**entity)
print(result)

[2020-04-10 13:25:53,802] -     INFO -       ontquery - interlex_client.py:962  - {'ilx_id': 'ILX:0738390', 'label': 'Offical label', 'type': 'term', 'definition': 'official definition', 'comment': 'helpful misc', 'superclass': '', 'synonyms': ['Encephalon', 'Cerebro'], 'add_existing_ids': None, 'delete_existing_ids': None, 'status': '0', 'cid': None}


QueryResult({'iri': 'http://uri.interlex.org/base/ilx_0738390', 'curie': 'ILX:0738390', 'label': 'Offical label', 'labels': (), 'definition': 'official definition', 'synonyms': ('Encephalon', 'Cerebro'), 'deprecated': None, 'predicates': {'comment': 'helpful misc'}, 'type': None, 'types': (), '_graph': None, 'source': <ontquery.plugins.services.interlex.InterLexRemote object at 0x11ba0a3d0>})


# PRODUCTION 
# BE CAREFUL PLEASE :)

In [27]:
# PRODUCTION 
from ilxutils.remotes import remote 
ixr = remote()
# BE CAREFUL :)

In [28]:
ixrt.ilx_cli.get_entity('ilx_0738390')

{'id': '661544',
 'orig_uid': '34142',
 'uid': '34142',
 'orig_cid': '0',
 'cid': '0',
 'ilx': 'ilx_0738390',
 'label': 'Offical label',
 'type': 'term',
 'definition': 'official definition',
 'comment': 'helpful misc',
 'version': '3',
 'status': '0',
 'display_superclass': '1',
 'orig_time': '1564695195',
 'time': '1586550353',
 'synonyms': [{'id': '1845765',
   'tid': '661544',
   'literal': 'Encephalon',
   'type': '',
   'time': '1586550353',
   'version': '3'},
  {'id': '1845766',
   'tid': '661544',
   'literal': 'Cerebro',
   'type': '',
   'time': '1586550353',
   'version': '3'}],
 'superclasses': [],
 'existing_ids': [{'id': '4972084',
   'tid': '661544',
   'curie': 'ILX:0738390',
   'iri': 'http://uri.interlex.org/base/ilx_0738390',
   'curie_catalog_id': '3885424',
   'version': '3',
   'time': '1586550353',
   'preferred': '1'}],
 'relationships': [],
 'mappings': [],
 'annotations': [],
 'ontologies': []}